In [2]:
import cv2
import numpy as np
import time

def create_background(cap):
    """Capture and return a static background frame"""
    print("Capturing background in 2 seconds. Please move out of frame...")
    time.sleep(2)
    _, background = cap.read()
    print("Background captured!")
    return background

def create_hsv_mask(frame, lower_bound, upper_bound):
    """Create a mask for the specified HSV color range"""
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    
    # Apply noise reduction
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8), iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, np.ones((3, 3), np.uint8), iterations=1)
    
    return mask

def main():
    # Initialize video capture
    cap = cv2.VideoCapture(0)
    
    # Allow the camera to warm up
    time.sleep(2)
    
    # Capture the background
    background = create_background(cap)
    
    # Define the color range for the "invisibility cloak" (red in this case)
    # You may need to adjust these values based on your lighting conditions
    lower_red = np.array([0, 120, 70])
    upper_red = np.array([10, 255, 255])
    
    # Second range for red (red wraps around in HSV)
    lower_red2 = np.array([170, 120, 70])
    upper_red2 = np.array([180, 255, 255])

    print("Setup complete. Hold up a red object to see the invisibility effect!")
    
    while True:
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            break
            
        # Create masks for both red ranges
        mask1 = create_hsv_mask(frame, lower_red, upper_red)
        mask2 = create_hsv_mask(frame, lower_red2, upper_red2)
        
        # Combine the masks
        mask = mask1 + mask2
        
        # Create the inverse mask
        mask_inv = cv2.bitwise_not(mask)
        
        # Use the mask to extract the foreground and background
        fg = cv2.bitwise_and(frame, frame, mask=mask_inv)
        bg = cv2.bitwise_and(background, background, mask=mask)
        
        # Combine foreground and background
        final_output = cv2.add(fg, bg)
        
        # Display the result
        cv2.imshow('Invisibility Cloak', final_output)
        
        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Clean up
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Capturing background in 2 seconds. Please move out of frame...
Background captured!
Setup complete. Hold up a red object to see the invisibility effect!
